In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# !pip install datasets
# !pip install evaluate

In [1]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import torch
import pandas as pd
import numpy as np
import os
import json

In [2]:
# ds = load_dataset("har1/MTS_Dialogue-Clinical_Note", streaming=False)
ds = load_dataset("ryanwtsai/MTS_Dialogue-Clinical_Note_OriginalTrainValSplit", streaming=False)
seq1id = 'dialogue'
seq2id = 'section_text'
ds

DatasetDict({
    train: Dataset({
        features: ['ID', 'section_header', 'section_text', 'dialogue', 'input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 1200
    })
    val: Dataset({
        features: ['ID', 'section_header', 'section_text', 'dialogue', 'input_ids', 'attention_mask', 'length', 'labels'],
        num_rows: 100
    })
})

In [3]:
device = 0 if torch.cuda.is_available() else -1

In [4]:
summarizer = pipeline("text2text-generation", model="har1/HealthScribe-Clinical_Note_Generator", device=device)

In [5]:
bertscore = evaluate.load('bertscore')
rouge = evaluate.load('rouge')
print(bertscore.__class__.__name__)
print(rouge.__class__.__name__)

BERTScore
Rouge


In [6]:
# data_split = 'train'
data_split = 'val'

ds_test = ds[data_split]
enc_seqs = []
predictions = []
references = []
for sdx in range(ds_test.num_rows):
    # print(f"Sample {sdx}")
    enc_seq = ds_test[sdx][seq1id]
    gnd_truth = ds_test[sdx][seq2id]
    pred = summarizer(enc_seq, truncation=True)
    predictions.append(pred[0]['generated_text'])
    references.append(gnd_truth)
    enc_seqs.append(enc_seq)

print("Finished predictions")

C:\ProgramData\anaconda3\envs\env_dl_cv_transformer_sentpiece\Lib\site-packages\transformers\models\bart\modeling_bart.py:597: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Finished predictions


In [7]:
result = bertscore.compute(predictions=predictions, references=references, model_type='distilbert-base-uncased')
result.pop('hashcode')
result = {data_split + "_" + f"bertscore_{k}": np.mean(np.array(v)) for k, v in result.items()}
metrics = result

In [8]:
result = rouge.compute(predictions=predictions, references=references)
result = {data_split + "_" + f"{k}": v for k, v in result.items()}
metrics.update(result)
print(metrics)

{'val_bertscore_precision': 0.8690398466587067, 'val_bertscore_recall': 0.9105064260959626, 'val_bertscore_f1': 0.8888216495513916, 'val_rouge1': 0.5421851318717354, 'val_rouge2': 0.3388843057001367, 'val_rougeL': 0.4634453874234585, 'val_rougeLsum': 0.5127617704569263}


In [9]:
# Save metrics

fname = r"predictions\har1_metrics.txt"

metrics_str = json.dumps(metrics, indent=0)
if not os.path.exists(fname):
    print(f"Creating file {fname}")
    with open(fname, "w") as file:
        file.write(metrics_str)
    print(f"File saved: {fname}")
else:
    print(f"Appending to file {fname}")
    with open(fname, "a") as file:
        file.write(metrics_str)
    print(f"File saved: {fname}")
    

Appending to file predictions\har1_metrics.txt
File saved: predictions\har1_metrics.txt


In [10]:
# Save predictions

fname2 = r"predictions\har1_" + data_split + ".json"
with open(fname2, "w") as file:
    json.dump(predictions, file)
print(f"Saved decoder output sequences to {fname2}")

Saved decoder output sequences to predictions\har1_val.json


In [11]:
with open(fname2, 'r') as file:
    decoded_outputs = json.load(file)
print(f"Loaded decoder output sequences from {fname2}")
print(type(decoded_outputs))
print(decoded_outputs[0])

Loaded decoder output sequences from predictions\har1_val.json
<class 'list'>
Symptoms: low back pain
Diagnosis: N/A
History of Patient: 26-year-old female, low-back pain for eight years, fell in ABC store 8 years ago, referred to Physical Therapy for treatment, received electrical stimulation and heat treatment, continuous pain over the last 8 years, worsened by another fall 4-5 days ago onto tailbone, seen Dr. X on January 10, 2009, follow-up appointment scheduled for February 10, 9, 2009
Plan of Action: Further evaluation and management of low- back pain with physical therapy, evaluation and referral to other doctors for evaluation and treatment



## Sanity check a few samples

In [12]:
data_split = 'train'
ds_test = ds[data_split]
fname = r"predictions\har1_" + data_split + ".json"

with open(fname, 'r') as file:
    decoded_outputs = json.load(file)

In [31]:
random_idx = np.random.randint(0, ds_test.num_rows, 10)
random_idx

array([ 535,  152, 1002,  741,  271,  490,  634,  679,  147,  841])

In [41]:
random_idx = np.random.randint(0, ds_test.num_rows, 10)
random_idx = [int(i) for i in random_idx]
# random_idx = [8]
for i in random_idx:
    print("Input:")
    print(ds_test[i][seq1id])
    print("")
    print("Ground truth:")
    print(ds_test[i][seq2id])
    print("")
    print("Predictions:")
    print(decoded_outputs[i])
    print("****************************************************************")

Input:
Doctor: Hi there, sir! How are you today? 
Patient: Hello! I am good.
Doctor: I would like to start with your family medical history today. What do you know about their medical history? 
Patient: My mother and father both had heart disease. Well, my mother had complication from her heart disease and that is how she passed.  My father was only in his forty's when he died. 
Doctor: I am so sorry the hear that. 
Patient: Thank you. I have two brothers, one whom I don't speak to very much anymore. I don't know if he has any health problems. My other brother is healthy with no issues. Both my uncles on my mother's side had polio, I think. 
Doctor: Tell me more about your uncles with polio. They both had polio? 
Patient: One of them had to wear crutches due to how bad his leg deformans were and then the other had leg deformities in only one leg.  I am fairly certain that they had polio.  
Doctor: Do you know of any other family member with neurological conditions?
Patient: No. None th